In [1]:
%matplotlib inline               
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('la data total classified.csv')

In [3]:
df.head()

,Daily Mean PM2.5 Concentration,Max Temp,Avg Dew Point,Max Wind Speed,Avg Humidity
0,3,88,32,5,26.0
1,3,68,46,6,67.5
2,3,71,44,5,61.0
3,3,68,50,6,63.5
4,3,55,46,8,79.5


In [4]:
df.describe()

,Daily Mean PM2.5 Concentration,Max Temp,Avg Dew Point,Max Wind Speed,Avg Humidity
count,3444.000000,3444.000000,3444.000000,3444.000000,3444.000000
mean,0.535134,75.954994,49.795296,8.855110,60.640534
std,0.702419,9.142370,10.425436,2.883481,13.194346
min,0.000000,51.000000,13.000000,0.000000,15.000000
25%,0.000000,69.000000,45.000000,8.000000,54.000000
50%,0.000000,76.000000,52.000000,9.000000,64.500000
75%,1.000000,82.000000,58.000000,10.000000,69.500000
max,3.000000,113.000000,68.000000,52.000000,99.000000


In [5]:
df.shape

(3444, 5)

In [6]:
X = df.drop('Daily Mean PM2.5 Concentration',axis=1)
y = np.asarray(df['Daily Mean PM2.5 Concentration'], dtype="|S6")

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
X_train.shape

(2755, 4)

In [9]:
X_test.shape

(689, 4)

In [10]:
y_train.shape

(2755,)

In [11]:
y_test.shape

(689,)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=1000)

parameter_space = {
    'hidden_layer_sizes': [(2, 2, 2), (4, 4, 4), (6, 6, 6)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [16]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(2, 2, 2), (4, 4, 4), (6, 6, 6)], 'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam'], 'alpha': [0.0001, 0.05], 'learning_rate': ['constant', 'adaptive']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [17]:
# Best parameter set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (6, 6, 6), 'learning_rate': 'constant', 'solver': 'adam'}
0.591 (+/-0.037) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (2, 2, 2), 'learning_rate': 'constant', 'solver': 'sgd'}
0.589 (+/-0.021) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (2, 2, 2), 'learning_rate': 'constant', 'solver': 'adam'}
0.583 (+/-0.005) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (2, 2, 2), 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.597 (+/-0.031) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (2, 2, 2), 'learning_rate': 'adaptive', 'solver': 'adam'}
0.597 (+/-0.034) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (4, 4, 4), 'learning_rate': 'constant', 'solver': 'sgd'}
0.599 (+/-0.027) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (4, 4, 4), 'learning_rate': 'constant', 'solver': 'adam'}
0.593 (+/-0.0

In [18]:
y_true, y_pred = y_test , clf.predict(X_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
             precision    recall  f1-score   support

       b'0'       0.66      0.86      0.74       395
       b'1'       0.48      0.35      0.40       222
       b'2'       0.50      0.05      0.09        63
       b'3'       0.00      0.00      0.00         9

avg / total       0.57      0.61      0.56       689



C:\Users\theda\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
